# TigerGraph Graph Data Science Library 101 - Community Detection Algorithm

This notebook shows the examples of using the most common community detection algorithms in TigerGraph Graph Science Library. More detailed explanations of these algorithms can be four in the official documentation (https://docs.tigergraph.com/graph-ml/current/community-algorithms/).  

## Step 1: Setting things up
- Connect and Load data
- Visualize the graph schema 
- Get basic stats, e.g., counts of nodes & edges


Connect and Load data: We will be loading the db connection info from our config.json


In [1]:
from pyTigerGraph.datasets import Datasets

dataset = Datasets("ldbc_snb")

Downloading:   0%|          | 0/286678171 [00:00<?, ?it/s]

In [2]:
from pyTigerGraph import TigerGraphConnection
import json

# Read in DB configs
with open('../config.json', "r") as config_file:
    config = json.load(config_file)

conn = TigerGraphConnection(
    host=config["host"],
    username=config["username"],
    password=config["password"],
)

In [3]:
conn.ingestDataset(dataset, getToken=config["getToken"])

---- Checking database ----
A graph with name ldbc_snb already exists in the database. Please drop it first before ingesting.


Visualize the graph schema: Adding visualization to would help better understand the algorithm results. In this tutorial, we will be using ipycytoscape to for visualization, and we will be loading the graph schema from TigerGraph Database. 


In [4]:
from pyTigerGraph.visualization import drawSchema

drawSchema(conn.getSchema(force=True))

CytoscapeWidget(cytoscape_layout={'name': 'circle', 'animate': True, 'padding': 1}, cytoscape_style=[{'selecto…

Get stats of the graph data (e.g., number of nodes and edges). 

In [5]:
vertices = conn.getVertexTypes()
for vertex in vertices:
    print("Node count: ({} : {}) ".format(vertex, conn.getVertexCount(vertex)))

Node count: (Comment : 2052169) 
Node count: (Post : 1003605) 
Node count: (Company : 1575) 
Node count: (University : 6380) 
Node count: (City : 1343) 
Node count: (Country : 111) 
Node count: (Continent : 6) 
Node count: (Forum : 90492) 
Node count: (Person : 9892) 
Node count: (Tag : 16080) 
Node count: (Tag_Class : 71) 


In [6]:
import pprint
print("Edges count: ")
pprint.pprint(conn.getEdgeCount())

Edges count: 
{'Container_Of': 1003605,
 'Container_Of_Reverse': 1003605,
 'Has_Creator': 3055774,
 'Has_Creator_Reverse': 3055774,
 'Has_Interest': 229166,
 'Has_Interest_Reverse': 229166,
 'Has_Member': 1611869,
 'Has_Member_Reverse': 1611869,
 'Has_Moderator': 90492,
 'Has_Moderator_Reverse': 90492,
 'Has_Tag': 3721417,
 'Has_Tag_Reverse': 3721417,
 'Has_Type': 16080,
 'Has_Type_Reverse': 16080,
 'Is_Located_In': 3073621,
 'Is_Located_In_Reverse': 3073621,
 'Is_Part_Of': 1454,
 'Is_Part_Of_Reverse': 1454,
 'Is_Subclass_Of': 70,
 'Is_Subclass_Of_Reverse': 70,
 'Knows': 180623,
 'Likes': 2190095,
 'Likes_Reverse': 2190095,
 'Reply_Of': 2052169,
 'Reply_Of_Reverse': 2052169,
 'Study_At': 7949,
 'Study_At_Reverse': 7949,
 'Work_At': 21654,
 'Work_At_Reverse': 21654}


## Step 2: Leveraging pyTigerGraph’s featurizer to run Community Detection algorithms

pyTIgerGraph provides a full suit of data science capabilities, and in this tutorial, we will showcase how to use featurizer to list out all available Centrality algorithms in our GDS library, and to run a few popular algorithms as an example.

In [6]:
feat = conn.gds.featurizer()

In [7]:
feat.listAlgorithms("Community")

Available algorithms for Community:
  connected_components:
    strongly_connected_components:
      01. name: tg_scc
  k_core:
    02. name: tg_kcore
  label_propagation:
    03. name: tg_label_prop
  local_clustering_coefficient:
    04. name: tg_lcc
  louvain:
    05. name: tg_louvain
  triangle_counting:
    06. name: tg_tri_count_fast
Call runAlgorithm() with the algorithm name to execute it


## tg_scc
A strongly connected component (SCC) is a subgraph such that there is a path from any vertex to every other vertex. A graph can contain more than one separate SCC. This SCC algorithm finds the maximal SCCs within a graph. (https://docs.tigergraph.com/graph-ml/current/community-algorithms/strongly-connected-components)

In [8]:
params =  {
    "v_type_set": ["Person"],
    "e_type_set": ["Knows"],
    "reverse_e_type_set": ["Knows"],
    "top_k_dist": 10,
    "print_limit": 10,
    "maximum_iteration": 50,
    "iter_wcc": 5,
    "print_results": False,
    "result_attribute": "tg_scc",
}

In [9]:
res = feat.runAlgorithm("tg_scc", params=params)

Altering graph schema to save results...
The job add_VERTEX_attr_WwpiRB completes in 2.405 seconds!
Installing and optimizing the queries, it might take a minute...
Queries installed successfully


In [10]:
df = conn.getVertexDataFrame("Person", limit=100_000)
display(df)

,v_id,id,first_name,last_name,gender,birthday,creation_date,location_ip,browser_used,speaks,email,tg_scc
0,2199023264346,2199023264346,Carlos,Gutierrez,male,1985-04-19 00:00:00,2010-03-10 12:52:07,201.220.206.82,Firefox,"[es, en]","[Carlos2199023264346@gmail.com, Carlos21990232...",1
1,21990232566217,21990232566217,Frank,Burns,male,1986-07-19 00:00:00,2011-10-31 06:33:04,14.1.16.88,Opera,"[vi, en]",[Frank21990232566217@hotmail.com],1
2,19791209304430,19791209304430,Paul,Fayed,male,1984-11-28 00:00:00,2011-08-05 06:38:58,62.12.112.179,Chrome,"[ar, fr, en]","[Paul19791209304430@gmx.com, Paul1979120930443...",1
3,17592186051479,17592186051479,Karim,Ben Dhifallah,male,1980-04-06 00:00:00,2011-05-11 04:24:17,193.95.38.162,Internet Explorer,"[ar, en]","[Karim17592186051479@gmx.com, Karim17592186051...",1
4,8796093024532,8796093024532,David,Jones,female,1988-09-30 00:00:00,2010-10-24 07:09:31,24.40.220.150,Internet Explorer,[en],[David8796093024532@gmx.com],1
...,...,...,...,...,...,...,...,...,...,...,...,...
9887,15393162794991,15393162794991,Deepak,Kapoor,male,1982-09-13 00:00:00,2011-04-03 11:47:23,49.156.125.15,Chrome,"[as, en]","[Deepak15393162794991@gmail.com, Deepak1539316...",1
9888,4398046511145,4398046511145,John,Kumar,male,1986-09-22 00:00:00,2010-05-19 01:14:14,27.116.33.147,Safari,"[gu, mr, en]","[John4398046511145@gmail.com, John439804651114...",1
9889,24189255821919,24189255821919,A.,Sharma,female,1988-07-15 00:00:00,2011-11-14 16:04:58,14.1.115.164,Internet Explorer,"[bn, kn, en]","[A.24189255821919@gmail.com, A.24189255821919@...",1
9890,26388279076505,26388279076505,Antonio,Alvarez,male,1984-11-23 00:00:00,2012-02-12 03:42:00,200.55.184.105,Firefox,"[es, en]","[Antonio26388279076505@gmail.com, Antonio26388...",1


In [12]:
df["tg_scc"].value_counts()

1           9163
11534594       1
11534629       1
11534638       1
10485762       1
            ... 
20971620       1
20971630       1
20971640       1
20971649       1
314            1
Name: tg_scc, Length: 730, dtype: int64

## tg_kcore
A k-core of a graph is a maximal connected subgraph in which every vertex is connected to at least 𝑘 vertices in the subgraph. (https://docs.tigergraph.com/graph-ml/current/community-algorithms/k-core-decomposition)

In [31]:
params = {
    "v_type": "Person",
    "e_type": "Knows",
    "k_min": 0,
    "k_max": -1,
    "print_results": False,
    "result_attribute": "tg_kcore",
    "file_path": "",
    "print_all_k": False,
    "show_shells": False
}

In [32]:
res = feat.runAlgorithm("tg_kcore", params = params)

In [33]:
df = conn.getVertexDataFrame("Person", limit=100_000)
display(df)
df["tg_kcore"].value_counts()

,v_id,id,first_name,last_name,gender,birthday,creation_date,location_ip,browser_used,speaks,email,tg_scc,tg_louvain,tg_kcore
0,2199023264346,2199023264346,Carlos,Gutierrez,male,1985-04-19 00:00:00,2010-03-10 12:52:07,201.220.206.82,Firefox,"[es, en]","[Carlos2199023264346@gmail.com, Carlos21990232...",1,32505856,20
1,21990232566217,21990232566217,Frank,Burns,male,1986-07-19 00:00:00,2011-10-31 06:33:04,14.1.16.88,Opera,"[vi, en]",[Frank21990232566217@hotmail.com],1,32505857,2
2,19791209304430,19791209304430,Paul,Fayed,male,1984-11-28 00:00:00,2011-08-05 06:38:58,62.12.112.179,Chrome,"[ar, fr, en]","[Paul19791209304430@gmx.com, Paul1979120930443...",1,32505858,21
3,17592186051479,17592186051479,Karim,Ben Dhifallah,male,1980-04-06 00:00:00,2011-05-11 04:24:17,193.95.38.162,Internet Explorer,"[ar, en]","[Karim17592186051479@gmx.com, Karim17592186051...",1,32505859,10
4,8796093024532,8796093024532,David,Jones,female,1988-09-30 00:00:00,2010-10-24 07:09:31,24.40.220.150,Internet Explorer,[en],[David8796093024532@gmx.com],1,32505860,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9887,15393162794991,15393162794991,Deepak,Kapoor,male,1982-09-13 00:00:00,2011-04-03 11:47:23,49.156.125.15,Chrome,"[as, en]","[Deepak15393162794991@gmail.com, Deepak1539316...",1,316,42
9888,4398046511145,4398046511145,John,Kumar,male,1986-09-22 00:00:00,2010-05-19 01:14:14,27.116.33.147,Safari,"[gu, mr, en]","[John4398046511145@gmail.com, John439804651114...",1,317,40
9889,24189255821919,24189255821919,A.,Sharma,female,1988-07-15 00:00:00,2011-11-14 16:04:58,14.1.115.164,Internet Explorer,"[bn, kn, en]","[A.24189255821919@gmail.com, A.24189255821919@...",1,318,5
9890,26388279076505,26388279076505,Antonio,Alvarez,male,1984-11-23 00:00:00,2012-02-12 03:42:00,200.55.184.105,Firefox,"[es, en]","[Antonio26388279076505@gmail.com, Antonio26388...",1,319,1


0     729
43    532
42    444
44    406
1     355
4     353
3     345
41    344
2     324
6     314
40    282
7     273
5     261
39    254
8     252
38    232
9     217
11    205
12    192
14    181
10    178
16    175
37    172
20    166
17    165
13    158
36    152
15    150
19    143
18    138
28    133
22    126
26    124
25    123
31    116
29    114
21    113
35    113
24    112
30    110
33    107
34    107
23    107
27    104
32    101
45     90
Name: tg_kcore, dtype: int64

## tg_louvain
The Louvain Method for community detection partitions the vertices in a graph by approximately maximizing the graph’s modularity score. (https://docs.tigergraph.com/graph-ml/current/community-algorithms/louvain)

In [20]:
params = {
    "v_type_set": ["Person"],
    "e_type_set": ["Knowns"],
    "weight_attribute": "",
    "maximum_iteration": 10,
    "result_attribute": "tg_louvain",
    "file_path": "",
    "print_stats": False
}

In [21]:
res = feat.runAlgorithm("tg_louvain", params = params)

Altering graph schema to save results...
The job add_VERTEX_attr_icyFZe completes in 45.064 seconds!
Installing and optimizing the queries, it might take a minute...
Queries installed successfully


[]

In [24]:
df = conn.getVertexDataFrame("Person", limit=100_000)
display(df)
df["tg_louvain"].value_counts()

,v_id,id,first_name,last_name,gender,birthday,creation_date,location_ip,browser_used,speaks,email,tg_scc,tg_louvain
0,2199023264346,2199023264346,Carlos,Gutierrez,male,1985-04-19 00:00:00,2010-03-10 12:52:07,201.220.206.82,Firefox,"[es, en]","[Carlos2199023264346@gmail.com, Carlos21990232...",1,32505856
1,21990232566217,21990232566217,Frank,Burns,male,1986-07-19 00:00:00,2011-10-31 06:33:04,14.1.16.88,Opera,"[vi, en]",[Frank21990232566217@hotmail.com],1,32505857
2,19791209304430,19791209304430,Paul,Fayed,male,1984-11-28 00:00:00,2011-08-05 06:38:58,62.12.112.179,Chrome,"[ar, fr, en]","[Paul19791209304430@gmx.com, Paul1979120930443...",1,32505858
3,17592186051479,17592186051479,Karim,Ben Dhifallah,male,1980-04-06 00:00:00,2011-05-11 04:24:17,193.95.38.162,Internet Explorer,"[ar, en]","[Karim17592186051479@gmx.com, Karim17592186051...",1,32505859
4,8796093024532,8796093024532,David,Jones,female,1988-09-30 00:00:00,2010-10-24 07:09:31,24.40.220.150,Internet Explorer,[en],[David8796093024532@gmx.com],1,32505860
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9887,15393162794991,15393162794991,Deepak,Kapoor,male,1982-09-13 00:00:00,2011-04-03 11:47:23,49.156.125.15,Chrome,"[as, en]","[Deepak15393162794991@gmail.com, Deepak1539316...",1,316
9888,4398046511145,4398046511145,John,Kumar,male,1986-09-22 00:00:00,2010-05-19 01:14:14,27.116.33.147,Safari,"[gu, mr, en]","[John4398046511145@gmail.com, John439804651114...",1,317
9889,24189255821919,24189255821919,A.,Sharma,female,1988-07-15 00:00:00,2011-11-14 16:04:58,14.1.115.164,Internet Explorer,"[bn, kn, en]","[A.24189255821919@gmail.com, A.24189255821919@...",1,318
9890,26388279076505,26388279076505,Antonio,Alvarez,male,1984-11-23 00:00:00,2012-02-12 03:42:00,200.55.184.105,Firefox,"[es, en]","[Antonio26388279076505@gmail.com, Antonio26388...",1,319


32505856    1
32505862    1
32505875    1
32505874    1
32505873    1
           ..
314         1
315         1
316         1
317         1
320         1
Name: tg_louvain, Length: 9892, dtype: int64

## tg_lcc

The local clustering coefficient of a vertex in a graph quantifies how close its neighbors are to being a complete graph. In a complete graph, every two distinct vertices are connected. (https://docs.tigergraph.com/graph-ml/current/community-algorithms/local-clustering-coefficient)

In [36]:
params = {
    "v_type": "Person",
    "e_type": "Knows",
    "top_k": 100,
    "print_results": False,
    "result_attribute": "tg_lcc",
    "file_path": "",
    "display_edges": False
}

In [37]:
res = feat.runAlgorithm("tg_lcc", params = params)

Altering graph schema to save results...
The job add_VERTEX_attr_eqlIsX completes in 37.602 seconds!


In [38]:
df = conn.getVertexDataFrame("Person", limit=100_000)
display(df)
df["tg_lcc"].value_counts()

,v_id,id,first_name,last_name,gender,birthday,creation_date,location_ip,browser_used,speaks,email,tg_scc,tg_louvain,tg_kcore,tg_lcc
0,2199023264346,2199023264346,Carlos,Gutierrez,male,1985-04-19 00:00:00,2010-03-10 12:52:07,201.220.206.82,Firefox,"[es, en]","[Carlos2199023264346@gmail.com, Carlos21990232...",1,32505856,0,0.16316
1,21990232566217,21990232566217,Frank,Burns,male,1986-07-19 00:00:00,2011-10-31 06:33:04,14.1.16.88,Opera,"[vi, en]",[Frank21990232566217@hotmail.com],1,32505857,1,1.00000
2,19791209304430,19791209304430,Paul,Fayed,male,1984-11-28 00:00:00,2011-08-05 06:38:58,62.12.112.179,Chrome,"[ar, fr, en]","[Paul19791209304430@gmx.com, Paul1979120930443...",1,32505858,0,0.15584
3,17592186051479,17592186051479,Karim,Ben Dhifallah,male,1980-04-06 00:00:00,2011-05-11 04:24:17,193.95.38.162,Internet Explorer,"[ar, en]","[Karim17592186051479@gmx.com, Karim17592186051...",1,32505859,0,0.15556
4,8796093024532,8796093024532,David,Jones,female,1988-09-30 00:00:00,2010-10-24 07:09:31,24.40.220.150,Internet Explorer,[en],[David8796093024532@gmx.com],1,32505860,0,0.15667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9887,15393162794991,15393162794991,Deepak,Kapoor,male,1982-09-13 00:00:00,2011-04-03 11:47:23,49.156.125.15,Chrome,"[as, en]","[Deepak15393162794991@gmail.com, Deepak1539316...",1,316,0,0.09959
9888,4398046511145,4398046511145,John,Kumar,male,1986-09-22 00:00:00,2010-05-19 01:14:14,27.116.33.147,Safari,"[gu, mr, en]","[John4398046511145@gmail.com, John439804651114...",1,317,0,0.13511
9889,24189255821919,24189255821919,A.,Sharma,female,1988-07-15 00:00:00,2011-11-14 16:04:58,14.1.115.164,Internet Explorer,"[bn, kn, en]","[A.24189255821919@gmail.com, A.24189255821919@...",1,318,0,0.20000
9890,26388279076505,26388279076505,Antonio,Alvarez,male,1984-11-23 00:00:00,2012-02-12 03:42:00,200.55.184.105,Firefox,"[es, en]","[Antonio26388279076505@gmail.com, Antonio26388...",1,319,1,0.00000


0.00000    1570
0.33333     319
0.16667     229
0.20000     222
0.14286     137
           ... 
0.12435       1
0.11739       1
0.08066       1
0.05652       1
0.09959       1
Name: tg_lcc, Length: 3152, dtype: int64